<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/GANS_Implementation_from_scratch_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>>You are on CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f">>>>{type(e)}: {e}\n>>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

def time_fmt(t: float = 128.912)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h}: min: {m:>02}: sec: {s:>05.2f}"
print(f">>>>time formating:\tplease wait......\n>>>>>time elapsed:\t{time_fmt()}")
  

Mounted at /content/drive
>>>>You are on CoLaB with torch version: 1.8.1+cu101
>>>>time formating:	please wait......
>>>>>time elapsed:	hrs: 0: min: 02: sec: 08.00


In [30]:
#In this notebook we are going to train a simple Generative Adversarial Network (GAN) on mnist digits

In [31]:
#The model architecture for the simple GAN consists of a generator and discriminator 
#generator will create fake data (random mnist images) and discriminator will classify
#the real image/data from the generated/fake data/images. This is to say in a simple language
#generator will try to fool the discriminator by producing almost similar data as the original
#At this point the error achieved is 50% (the discriminator has 50% chance of identify the fake images from the real)

In [32]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import math, random, time
from torch.utils.tensorboard import SummaryWriter


In [33]:
#Set random seeds for reproducability
seed = 1234
random.seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [34]:
#The following class will build a discriminator network
class Discriminator(nn.Module):
  def __init__(self, in_features):
    super(Discriminator, self).__init__()
    self.discriminator = nn.Sequential(
        nn.Linear(in_features = in_features, out_features = 128),
        nn.LeakyReLU(0.01),
        nn.Linear(in_features = 128, out_features = 1),
        nn.Sigmoid())
    
  def forward(self, input_tensor):
    x = self.discriminator(input_tensor)
    return x

In [35]:
#This class will build the generator network
class Generator(nn.Module):
  def __init__(self, z_dim, x_dim):
    super(Generator, self).__init__()
    self.generator = nn.Sequential(
        nn.Linear(in_features = z_dim, out_features = 256),
        nn.LeakyReLU(0.01),
        nn.Linear(in_features = 256, out_features = x_dim),
        nn.Tanh())
    
  
  def forward(self, input_tensor):
    x = self.generator(input_tensor)
    return x

In [36]:
#Hyperparameters for the Model:
batch_size = 32
EPOCHS = 100
x_dim = 784 #mnist image shape when flattened (28*28*28)
z_dim = 64 # dimension for the random noise
learning_rate = 3e-4


In [37]:
#Get the data from torchvission, preprocess using transform and splits into batches using DataLoader
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5,))])
mnist = datasets.MNIST(root = "mnist_data/", transform=transforms, download = True)
loader = DataLoader(dataset = mnist, batch_size = batch_size, shuffle = True)
x_mnist, y_mnist = next(iter(loader))
print(f">>>>x_mnist_shape: {x_mnist.shape}\ty_mnist_shape: {y_mnist.shape}")

>>>>x_mnist_shape: torch.Size([32, 1, 28, 28])	y_mnist_shape: torch.Size([32])


In [38]:
#Get the loss and optimizers objects
discriminator = Discriminator(x_dim).to(device = device)
generator = Generator(z_dim, x_dim).to(device = device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device = device)
writer_real = SummaryWriter(f"gans/real")
writer_fake = SummaryWriter(f"gans/fake")
loss_obj = nn.BCELoss()
disc_optimizer = optim.Adam(params = discriminator.parameters(), lr = learning_rate)
gen_optimizer = optim.Adam(params = generator.parameters(), lr = learning_rate)


In [39]:
#To train this model, discriminator and generator are trained separately:
#For discriminator we need to maximize [log(D(x)) + log(1 - D(G(z)))]
#For generator we need to maximize [log(D(G(z)))]

In [40]:
#The training loop
step = 0
for epoch in range(EPOCHS):
  print(f"\n>>>>Training starts for epoch: {epoch + 1}\tplease wait........")
  for idx, (real,_) in enumerate(tqdm(loader)):
    real = real.view(-1, 784).to(device = device) 
    batch_size = real.shape[0]#grab the batch size
    noise = torch.randn((batch_size, z_dim)).to(device = device) #Create a noisy image/data
    fake = generator(noise)#Generate a very noisy image
    disc_real = discriminator(real).view(-1)#output of a real image on discriminator
    disc_loss_real = loss_obj(disc_real, torch.ones_like(disc_real)) #Loss when we classify a real image
    disc_fake = discriminator(fake).view(-1) #output of discriminator for the fake image
    disc_loss_fake = loss_obj(disc_fake, torch.zeros_like(disc_fake))#Loss when we classify a fake image
    disc_loss = (disc_loss_real + disc_loss_fake) / 2 #Take the average of the loss(discriminator)
    discriminator.zero_grad()
    disc_loss.backward(retain_graph = True)# backward pass for the discriminator
    disc_optimizer.step()
    #Train the generator
    gen_out = discriminator(fake).view(-1)# output from generator when passing a fake image
    loss_gen = loss_obj(gen_out, torch.ones_like(gen_out)) #Make generator smart to fool a discriminator
    generator.zero_grad()
    loss_gen.backward()#backward pass for generator
    gen_optimizer.step()#gradient descent with adam opt for generator
    if idx % 100 == 0:
      print(f"\n>>>>Epoch: {epoch + 1}/{EPOCHS}")
      print(f"\n>>>>Generator Loss: {loss_gen:.4f} | Generator PPL: {math.exp(loss_gen):7.4f}")
      print(f"\n>>>>Discriminator Loss: {disc_loss:.4f} | Discriminator PPL: {math.exp(disc_loss):7.4f}")
      #Writting the fake and real images/data to the prepared directories
      with torch.no_grad():
        fake_data = generator(fixed_noise).reshape(-1,1,28,28)
        real_data = real.reshape(-1, 1, 28,28)
        img_grid_fake = torchvision.utils.make_grid(fake_data, normalize = True)
        img_grid_real = torchvision.utils.make_grid(real_data,normalize = True)
        writer_real.add_image('image_real', img_grid_real, global_step = step)
        writer_fake.add_image('image_fake', img_grid_fake,global_step = step)
        step+=1



>>>>Training starts for epoch: 1	please wait........


  0%|          | 9/1875 [00:00<00:21, 85.83it/s]


>>>>Epoch: 1/100

>>>>Generator Loss: 0.7116 | Generator PPL:  2.0373

>>>>Discriminator Loss: 0.6287 | Discriminator PPL:  1.8752


  6%|▌         | 112/1875 [00:00<00:15, 113.10it/s]


>>>>Epoch: 1/100

>>>>Generator Loss: 0.8824 | Generator PPL:  2.4166

>>>>Discriminator Loss: 0.4656 | Discriminator PPL:  1.5930


...

 92%|█████████▏| 1718/1875 [00:15<00:01, 108.91it/s]


>>>>Epoch: 100/100

>>>>Generator Loss: 0.9989 | Generator PPL:  2.7152

>>>>Discriminator Loss: 0.5394 | Discriminator PPL:  1.7150


 97%|█████████▋| 1820/1875 [00:16<00:00, 106.69it/s]


>>>>Epoch: 100/100

>>>>Generator Loss: 0.8939 | Generator PPL:  2.4446

>>>>Discriminator Loss: 0.6576 | Discriminator PPL:  1.9301


100%|██████████| 1875/1875 [00:17<00:00, 107.88it/s]
